In [1]:
import graphlab as gl
import matplotlib.pyplot as plt
import os


In [2]:
def Edges(P):
    points = sorted(set(P))
    def cross(o, a, b):
        return (a[0] - o[0]) * (b[1] - o[1]) - (a[1] - o[1]) * (b[0] - o[0])
    lower = []
    for count in points:
            while len(lower) >= 2 and cross(lower[-2], lower[-1], count) <= 0:
                lower.pop()
            lower.append(count)
    upper = []
    for count in reversed(points):
            while len(upper) >= 2 and cross(upper[-2], upper[-1], count) <= 0:
                upper.pop()
            upper.append(count)
    shape = lower[:-1] + upper[:-1]
    shapex,shapey = zip(*shape)
    return shape

In [3]:
def Area(shape):
    Area = 0
    Add = 0
    Sub = 0
    for a in range(len(shape)):
        if a != (len(shape)-1):
            Add = Add + shape[a][0]* shape[a+1][1]
            Sub = Sub + shape[a][1]* shape[a+1][0]
        else:
            Add = Add + shape[a][0]* shape[0][1]
            Sub = Sub + shape[a][1]* shape[0][0]
    Area = (((Add - Sub)**2)**0.5)* 0.5 / 1000000
    return Area

In [62]:
def Cluster_per_Area(filename):
    
    Raw_Image = gl.SFrame.read_csv(filename, delimiter = '	',header = True)

#shift points so the min is (0,0)
    Sauce = gl.SFrame({'X': Raw_Image['Xc']-min(Raw_Image['Xc']),'Y': Raw_Image['Yc']-min(Raw_Image['Yc'])})
    
#Runs a loose parameter DBSCAN on the points
    Image_C = gl.dbscan.create(Sauce, radius = 30, min_core_neighbors = 20)

#Give each point its own identification number
    Sauce = Sauce.add_row_number('row_id')

#Add the cluster identity to each point 
    Image_Pts = Sauce.join(Image_C['cluster_id'], on='row_id', how='left')

    Image_Pts = Image_Pts[Image_Pts['cluster_id']>=0] #Removes all non-clustered points

                #determining the treatment and the identity from the file name
    if filename.find('++') > 0:
        Treat = 'Antagonist'
                #Treat = 'CytD'
    elif filename.find('+-')>0:
        Treat = 'Nicotine'
                #Treat = 'mbCD'
    else :
        Treat = "Control"

    if filename.find('E')>0:
        Subject= 'Extension'
    else:
        Subject ='Varicosity'    
    
#Image_Pts.remove_columns(['row_id','cluster_id','type'])
    x,y = Image_Pts['X'],Image_Pts['Y']
    P = zip(x,y)
    if len(P) >= 1:   
        shape = Edges(P)
        Area_Var = Area(shape)
        Num_C = int((Image_Pts.sort('cluster_id', ascending = False))[1]['cluster_id'])
        Clusters_Per_Area = Num_C / Area_Var
        Result = gl.SFrame({'Cluster per Area':[Clusters_Per_Area],'Clusters':[Num_C],'Area':[Area_Var],
                            'File':[filename],'Treatment':[Treat],'Subject':[Subject]})
    else:
         Result = gl.SFrame({'Cluster per Area':[0.0],'Clusters':[0],'Area':[0.0],
                            'File':[filename],'Treatment':[Treat],'Subject':[Subject]})
    return Result



In [ ]:
plt.scatter(Sauce['X'],Sauce['Y'])
plt.scatter(shapex,shapey,edgecolors ='red')
#plt.axis([0,10000,0,8000])
plt.show()


In [63]:
Total = gl.SFrame({'Cluster per Area':gl.SArray(dtype = float),'Clusters':gl.SArray(dtype = int),
                   'Area':gl.SArray(dtype = float),
                   'File':gl.SArray(dtype = str),'Treatment':gl.SArray(dtype = str),
                   'Subject':gl.SArray(dtype = str)})
for f in os.listdir('Cluster_Data'):     #loads up every txt file in the cluster_data dir but sources from main dir
    Total = Total.append(Cluster_per_Area(f))    #makes master list

Total = Total.sort(['Subject',"Treatment"])
Total.export_csv(filename = 'Cluster_per_area.txt', delimiter = '	')



Finished parsing file C:\Users\jdr248\160512 +- E 002.txt

Parsing completed. Parsed 100 lines in 0.029003 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[float,float,str,str,str,str,float,str,str,long,long,long,long,long,long]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file C:\Users\jdr248\160512 +- E 002.txt

Parsing completed. Parsed 6553 lines in 0.051202 secs.

...

+-----------------------------+

| 7219                        |

| 0                           |

+-----------------------------+

[INFO] graphlab.toolkits.clustering.dbscan: Processing boundary points.


In [61]:

filename = '160512 +- E 015.txt'    
Raw_Image = gl.SFrame.read_csv(filename, delimiter = '	',header = True)

#shift points so the min is (0,0)
Sauce = gl.SFrame({'X': Raw_Image['Xc']-min(Raw_Image['Xc']),'Y': Raw_Image['Yc']-min(Raw_Image['Yc'])})
    
#Runs DBSCAN on the points
Image_C = gl.dbscan.create(Sauce, radius = 30, min_core_neighbors = 20)

#Give each point its own identification number
Sauce = Sauce.add_row_number('row_id')

#Add the cluster identity to each point 
Image_Pts = Sauce.join(Image_C['cluster_id'], on='row_id', how='left')

Image_Pts = Image_Pts[Image_Pts['cluster_id']>=0] #Removes all non-clustered points

#Image_Pts.remove_columns(['row_id','cluster_id','type'])
x,y = Image_Pts['X'],Image_Pts['Y']
P = zip(x,y)
print len(P)

            #determining the treatment and the identity from the file name
if filename.find('++') > 0:
    Treat = 'Antagonist'
            #Treat = 'CytD'
elif filename.find('+-')>0:
    Treat = 'Nicotine'
            #Treat = 'mbCD'
else :
    Treat = "Control"

if filename.find('E')>0:
    Subject= 'Extension'
else:
    Subject ='Varicosity'

if len(P) >= 1:   
    shape = Edges(P)
    Area_Var = Area(shape)
    Num_C = int((Image_Pts.sort('cluster_id', ascending = False))[1]['cluster_id'])
    Clusters_Per_Area = Num_C / Area_Var
    Result = gl.SFrame({'Cluster per Area':[Clusters_Per_Area],'Clusters':[Num_C],'Area':[Area_Var],
                            'File':[filename],'Treatment':[Treat],'Subject':[Subject]})
else:
    Result = gl.SFrame({'Cluster per Area':[0],'Clusters':[0],'Area':[0],
                            'File':[filename],'Treatment':[Treat],'Subject':[Subject]})
print Result



Finished parsing file C:\Users\jdr248\160512 +- E 015.txt

Parsing completed. Parsed 100 lines in 0.012001 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[float,float,str,str,str,str,float,str,str,long,long,long,long,long,long]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file C:\Users\jdr248\160512 +- E 015.txt

Parsing completed. Parsed 399 lines in 0.012001 secs.

...

+-----------------------------+

| 0                           |

+-----------------------------+

[INFO] graphlab.toolkits.clustering.dbscan: Processing boundary points.


0
+------+------------------+----------+---------------------+-----------+
| Area | Cluster per Area | Clusters |         File        |  Subject  |
+------+------------------+----------+---------------------+-----------+
|  0   |        0         |    0     | 160512 +- E 015.txt | Extension |
+------+------------------+----------+---------------------+-----------+
+-----------+
| Treatment |
+-----------+
|  Nicotine |
+-----------+
[1 rows x 6 columns]



()